In [1]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator
import os
import time
import torch, torch.nn as nn
from torch.nn.utils import prune

In [2]:
version = 'v2'
# weight = os.path.join('models', version, 'pruned.pt')
weight = os.path.join('models', version, 'best.pt')
weight_abspath = os.path.abspath(weight)

print(weight_abspath)

model = YOLO(weight, task='detect')
# metrics = model.val(data=os.path.abspath('../merge_dataset/data.yaml'), plots=True, task='detect', conf=0.35)
# model.export(format='onnx', int8=True, dynamic=True, simplify=True)
# model.export(format='openvino', int8=True)

c:\Users\sorra\projects\mlproj\gun-detection-system\models\v2\best.pt
Ultralytics YOLOv8.1.30 🚀 Python-3.12.2 torch-2.2.1+cpu CPU (AMD Ryzen 5 3550H with Radeon Vega Mobile Gfx)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning C:\Users\sorra\projects\mlproj\merge_dataset\valid\labels.cache... 1607 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1607/1607 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 1821. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [05:35<00:00,  3.32s/it]


                   all       1607       1821      0.946       0.92      0.953       0.76
Speed: 1.9ms preprocess, 195.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val


In [7]:
# print(metrics.box.map)    # map50-95 standard benchmark
# print(metrics.box.map50)  # map50
# print(metrics.box.map75)  # map75
# print(metrics.box.maps)

0.7595778086504898
0.9526791491582957
0.8636578261301457
[    0.75958]


In [3]:
# def sparsity(model):
#     # Return global model sparsity
#     a, b = 0, 0
#     for p in model.parameters():
#         a += p.numel()
#         b += (p == 0).sum()
#     return b / a

# pruning_param = 0.8

# for name, m in model.model.named_modules():
#     if isinstance(m, nn.Conv2d) or isinstance(m, torch.nn.Linear):
#         prune.l1_unstructured(m, name='weight', amount=pruning_param)  # prune
#         prune.remove(m, 'weight')  # make permanent
# print(f'Model pruned to {sparsity(model.model):.3g} global sparsity')

# ckpt = {
#     'model': model.model,
#     'train_args': {},  # save as dict
# }

# torch.save(ckpt, os.path.abspath(os.path.join('models', version, 'pruned.pt')))

In [4]:
# cap = cv2.VideoCapture(0)
# # cap = cv2.VideoCapture('sample.mp4')

# # Initialize FPS calculation
# prev_time = 0

# while cap.isOpened():
#     ret, frame = cap.read()

#     current_time = time.time()

#     results = model.predict(frame, conf=0.35, stream=True, vid_stride=5, stream_buffer=True)

#     for result in results:

#         # https://stackoverflow.com/questions/75324341/yolov8-get-predicted-bounding-box
#         annotator = Annotator(frame)

#         boxes = result.boxes
#         conf_arr = boxes.conf.numpy()
#         conf = 0
#         for box in boxes:
#             b = box.xyxy[0]
#             c = box.cls
            
#             if len(conf_arr) > 0:
#                 conf = conf_arr[0]
#             annotator.box_label(b, model.names[int(c)]+" "+f'{conf:.2f}')
    
#     frame = annotator.result() 

#     # Calculate FPS
#     fps = 1 / (current_time - prev_time)
#     prev_time = current_time

#     # Display FPS on top-left corner of the window
#     cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

#     cv2.imshow('Gun detection', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()